In [23]:
import mesa
import numpy as np
from numpy import random

In [ ]:
TIMESTEP = 15 # minutes = 1 tick
STATES = ['idle', 'charging', 'pickup', 'dropoff', 'going to recharge'] # car states
SPEED = 8 # miles per tick
CHARGING = 80 # kW outputted per hour
FUEL = 1 / 5 # kWh used to drive 1 mile

In [28]:
def manhattan_distance(pos1, pos2):
    # MANHATTAN DISTANCE from POS1 to POS2
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

def random_pos(x, y):
    # RANDOM POSITION on the X by Y board
    return (random.randint(x - 1), random.randint(y - 1))

In [32]:
class Car(mesa.Agent):
    def __init__(self, id, model, pos, capacity = 80, battery = 1):
        # INITIALIZE CAR agent at position POS in MODEL, with battery capacity CAPACITY in kWh,
        # and battery level BATTERY
        super().__init__(id, model)
        model.place(self, pos)
        self.battery = battery
        self.capacity = capacity
        self.state = 'idle'
        self.dest = None
        self.pickup = None
        self.charging_time = 0 # minutes to finish charging to 100

    def go_charge(self):
        self.state = 'going to recharge'
        self.dest = self.find_closest_station()

    def start_new_itinerary(self, pickup, dropoff):
        dist_to_pickup = manhattan_distance(self.pos, pickup)
        dist_to_dest = manhattan_distance(pickup, dropoff)
        closest_station = self.find_closest_station(dropoff)
        dist_to_charging = manhattan_distance(dropoff, closest_station)
        total_dist = dist_to_pickup + dist_to_dest + dist_to_charging
        battery_needed = total_dist * FUEL

        if self.battery * self.capacity >= battery_needed:
            self.state = 'pickup'
            self.pickup = pickup
            self.dropoff = dropoff
        else:
            self.go_charge()

    def step(self):
        if self.charging_time:
            self.decrement_charging()
            return
        elif self.state == 'idle':
            return
        else:
            dx = abs(self.pos[0] - self.dest[0])
            dy = abs(self.pos[1] - self.dest[1])
            if SPEED >= dx + dy:
                self.move(self.dest)
                self.update_state()
            elif SPEED <= dx:
                self.move(dx, 0)
            elif SPEED <= dy:
                self.move(0, dy)
            else:
                remainder = SPEED - dx
                assert remainder > 0, "remainder not greater than 0?!"
                self.move(dx, remainder)
                
    def update_state(self):
        state = self.state
        match state:
            case 'pickup':
                self.state = 'dropoff'
                self.pickup = None
            case 'going to recharge':
                self.start_charging()
            case 'dropoff':
                self.state = 'idle'
                # self.decide ?
            case 'charging':
                self.state = 'idle'

    def decrement_charging(self):
        self.charging_time = max(self.charging_time - TIMESTEP, 0)
        if not self.charging_time:
            self.update_state()
    
    def start_charging(self):
        # Begin charging process
        self.state = 'charging'
        battery_needed = (1 - self.battery) * self.capacity
        self.charging_time = ((battery_needed / CHARGING * 60) // 15 + 1)* 15

    def move(self, dx, dy):
        # MOVE DX units in the x axis towards destination, DY units in y axis towards destination
        # updates battery accordingly. DY, DX positive for simplicity.
        assert dx > 0 and dy > 0, "DX, DY not > 0!"
        dx *= -1 if self.pos[0] > self.dest[0] else 1
        dy *= -1 if self.pos[1] > self.dest[1] else 1
        new_pos = (self.pos[0] + dx, self.pos[1] + dy)
        self.model.grid.move_agent(self, new_pos)
        return


class Charger(mesa.Agent):
    def __init__(self, id, model, pos, capacity = np.inf, rate = 1,):
        super().__init__(id, model)
        model.grid.place_agent(self, pos)
        model.
        self.capacity = capacity
        self.rate = rate

class City(mesa.Model):
    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.grid = mesa.space.MultiGrid(rows, cols, False)
        self.schedule = mesa.time.BaseScheduler(self)
        self.charging_stations = {}
    
    def place(self, car, pos):
        # Places a car on the grid
        self.grid.place_agent(car, pos)
        self.schedule.add(car)
    
    def step(self):
        self.schedule.step()
    
    def print(self):
        for c in range(self.cols):
            s = ''
            for r in range(self.rows):
                car = self.grid[r][c]
                s += ' C ' if car else ' . '
            print(s)


In [33]:
num_cars = 5
x, y = 25, 25
generate_pos = lambda : random_pos(x, y)
sf = City(x, y)
cars = []
for i in range(num_cars):
    pos = generate_pos()
    cars.append(Car(i, sf, pos))
    pos = generate_pos()
    Charger(num_cars + i, sf, pos)
sf.print()


 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  C  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  C  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 C  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  .  . 
 .  .  .  . 